<a href="https://colab.research.google.com/github/guidobarra/pyhton-GPU/blob/main/BarraQuelca_GuidoAlberto_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 Introducción

El siguiente ejemplo se realizará la búsqueda de un elemento dentro de un array.
En el mundo de la informática hay varios algoritmos de búsqueda como por ejemplo la búsqueda binaria en el cual los elementos del array deben estar ordenados para realizar el algoritmo, otro algoritmo es la búsqueda secuencial el más simple de todos. La búsqueda de un objeto o palabra es algo muy utilizado, un ejemplo es el algoritmo de búsqueda de Google, buscar una palabra o secuencia de caracteres dentro de un archivo, buscar claves dentro de una tabla de base de datos.

EL objetivo es enseñar el funcionamiento del Lenguaje Python, CUDA y buscar un número dentro de un array. El ejemplo es ilustrativo para entender los multi hilos de la GPU de una dimensión y ver la performance utilizando la GPU y CPU.

---
# 2 Armado del ambiente
Toma los parametros de cantidad de elementos del array y el numero que debe buscar, deja disponible al contexto de ejecuciòn del cuaderno colab.

In [1]:
#@title # 2.1 Parámetros de ejecución
#@markdown ### Especifique las dimeciones de la matrices:

try: 
  cant_element =  100000000#@param {type:"integer"}
  encontrar_numero =  1000000000#@param {type:"integer"}
except Exception as e:
  print("Error de ingresar los parametros")
  print("Error debido a: ", e.__class__)
  print(e)






---
## 2.2 Validar las parametros

In [3]:

if cant_element < 0 :
  print("ERROR LA CANTIDAD DE ELEMENTOS NO PUEDE SER MENOR QUE CERO")

---
## 2.3 Instala en el cuaderno el módulo CUDA de Python.

In [6]:
!pip install pycuda

     |████████████████████████████████| 1.6MB 8.7MB/s 
     |████████████████████████████████| 71kB 10.0MB/s 
     |████████████████████████████████| 81kB 11.7MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=621008 sha256=d51f101014ef7dea5a347020e89ebee1d7540ddf1d871e26190cd0ac7718eee5
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.3-py2.py3-none-any.whl size=61374 sha256=c8d1a0a1d99bf421b05cbf41b23b09d8fe9b81f9d539e66f7db5e5fea6243bb1
  Stored in directory: /root/.cache/pip/wheels/af/c7/81/a22edb90b0b09a880468b2253bb1df8e9f503337ee15432c64
Successfully built pycuda pytools


---
# 3 Desarrollo
Ejecución del algoritmo de busqueda utilizando la CPU para el desarrollo en su forma secuencual y la GPU para el desarrollo en su forma paralela.

## 3.1 Desarrollo Secuencial
Ejecución del algoritmo de busqueda de forma secuencial utilizando la CPU.

In [4]:
try:
  %matplotlib inline
  from datetime import datetime
  tiempo_total = datetime.now()
  import numpy

  # --------------------------------------------
  # Definición de función que transforma el tiempo en  milisegundos
  tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0
  # --------------------------------------------

  tiempo_definicion_vector = datetime.now()

  # CPU - Defino la memoria del vector de n elementos en cpu.
  vector = numpy.array(range(-cant_element, cant_element, 2))
  vector = vector.astype(numpy.int32())

  tiempo_definicion_vector = datetime.now() - tiempo_definicion_vector

  tiempo_find_secuencial = datetime.now()

  position = -1
  for i in range(cant_element):
    if vector[i] == encontrar_numero:
      position = i
      break

  tiempo_find_secuencial = datetime.now() - tiempo_find_secuencial

  tiempo_total = datetime.now() - tiempo_total

  print("\n**ENCONTRAR UN ELEMENTO DENTRO DE UN VECTOR UTILIZANDO LA CPU**\n")
  print("Cantidad de elementos del vector: ", cant_element)
  print("Numero a encontrar: ", encontrar_numero)
  print("Vector: \n", vector)
  if position == -1:
    print("No se encontro el numero: ", encontrar_numero)
  else:
    print("Se encontro el numero: ", encontrar_numero)
    print("Posicion dentro del vector es: ", position)

  print("Tiempo de definicion del vector: ", tiempo_en_ms( tiempo_definicion_vector ), "[ms]" )
  print("Tiempo CPU  : ", tiempo_en_ms( tiempo_find_secuencial ), "[ms]" )
  print("Tiempo TOTAL: ", tiempo_en_ms( tiempo_total ), "[ms]" )
except Exception as e:
  print("Oops Ocurrio una error!")
  print("Error debido a: ", e.__class__)
  print(e)


**ENCONTRAR UN ELEMENTO DENTRO DE UN VECTOR UTILIZANDO LA CPU**

Cantidad de elementos del vector:  100000000
Numero a encontrar:  1000000000
Vector: 
 [-100000000  -99999998  -99999996 ...   99999994   99999996   99999998]
No se encontro el numero:  1000000000
Tiempo de definicion del vector:  17484.785 [ms]
Tiempo CPU  :  153227.442 [ms]
Tiempo TOTAL:  170712.311 [ms]


## 3.2 Desarrollo Paralelo
Ejecución del algoritmo de busqueda de forma paralelo utilizando la GPU.

In [7]:
try:
  %matplotlib inline
  from datetime import datetime
  tiempo_total = datetime.now()
  
  import numpy
  import pycuda.driver as cuda
  import pycuda.autoinit
  from   pycuda.compiler import SourceModule
  
  # --------------------------------------------
  # Definición de función que transforma el tiempo en  milisegundos
  tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0
  # --------------------------------------------
  
  tiempo_definicion_vector = datetime.now()
  
  # CPU - Defino la memoria del vector de n elementos en cpu.
  vector = numpy.array(range(-cant_element, cant_element, 2))
  vector = vector.astype(numpy.int32())
  
  # CPU - Defino la memoria del vector position_v de 1 elemento en cpu.
  position_v    = numpy.random.randn(1)*0
  position_v[0] = -1
  position_v    = position_v.astype(numpy.int32())
  
  tiempo_definicion_vector = datetime.now() - tiempo_definicion_vector
  
  tiempo_reserva_memoria_GPU = datetime.now()
  # CPU - reservo la memoria GPU.
  vector_Gpu = cuda.mem_alloc(vector.nbytes)
  position_v_Gpu = cuda.mem_alloc(position_v.nbytes)
  tiempo_reserva_memoria_GPU = datetime.now() - tiempo_reserva_memoria_GPU
  
  tiempo_copia_memoria_GPU = datetime.now()
  # GPU - Copio la memoria al GPU.
  cuda.memcpy_htod(vector_Gpu, vector)
  cuda.memcpy_htod(position_v_Gpu, position_v)
  tiempo_copia_memoria_GPU = datetime.now() - tiempo_copia_memoria_GPU
  
  #CPU - Defino la funcion kernel que ejecutará en GPU
  module = SourceModule("""
  __global__ void find(int tam, int number_find, int *vector, int *position)
  {
      // Calculo las coordenadas del Thread en dos dimensiones.
      int idx = threadIdx.x + blockIdx.x*blockDim.x;
  
      // Verifico que los Thread, esten dentro de las dimensiones del vector
      if( idx < tam ) {
        if (vector[idx] == number_find && position[0] == -1) {
          position[0] = idx;
        }
      }
  }
  
  """)
  
  # CPU - Genero la función kernel.
  kernel = module.get_function("find")
  
  # GPU - Ejecuta el kernel.
  # TODO: Falta consultar limites del GPU, para armar las dimensiones correctamente.
  dim_hilo = 256
  dim_bloque = numpy.int( (cant_element+dim_hilo-1) / dim_hilo )
  print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )
  
  tiempo_find_paralelo = datetime.now()
  
  kernel( numpy.int32(cant_element),
          numpy.int32(encontrar_numero),
          vector_Gpu,
          position_v_Gpu,
          block=( dim_hilo, 1, 1 ),
          grid=(dim_bloque, 1,1) )
  
  tiempo_find_paralelo = datetime.now() - tiempo_find_paralelo
  
  # GPU - Copio el resultado desde la memoria GPU.
  cuda.memcpy_dtoh( position_v, position_v_Gpu )
  tiempo_total = datetime.now() - tiempo_total
  
  print("\n**ENCONTRAR UN ELEMENTO DENTRO DE UN VECTOR UTILIZANDO LA GPU**\n")
  print("Cantidad de elementos del vector: ", cant_element)
  print("Numero a encontrar: ", encontrar_numero)
  print("Vector: \n", vector)
  if position == -1 or position_v[0] == -1:
    print("No se encontro el numero: ", encontrar_numero)
  else:
    print("Se encontro el numero: ", encontrar_numero)
    print("Posicion dentro del vector es: ", position_v[0])
    print("Posicion dentro del vector es: ", position)

  print("Tiempo de definicion del vector: ", tiempo_en_ms( tiempo_definicion_vector ), "[ms]" )
  print("Tiempo de reserva del vector: ", tiempo_en_ms( tiempo_reserva_memoria_GPU ), "[ms]" )
  print("Tiempo de copia del vector: ", tiempo_en_ms( tiempo_copia_memoria_GPU ), "[ms]" )
  print("Tiempo GPU  : ", tiempo_en_ms( tiempo_find_paralelo ), "[ms]" )
  print("Tiempo TOTAL: ", tiempo_en_ms( tiempo_total ), "[ms]" )
except Exception as e:
  print("Oops Ocurrio una error!")
  print("Error debido a: ", e.__class__)
  print(e)

Thread x:  256 , Bloque x: 390625

**ENCONTRAR UN ELEMENTO DENTRO DE UN VECTOR UTILIZANDO LA GPU**

Cantidad de elementos del vector:  100000000
Numero a encontrar:  1000000000
Vector: 
 [-100000000  -99999998  -99999996 ...   99999994   99999996   99999998]
No se encontro el numero:  1000000000
Tiempo de definicion del vector:  17964.016 [ms]
Tiempo de reserva del vector:  0.933 [ms]
Tiempo de copia del vector:  85.178 [ms]
Tiempo GPU  :  1.389 [ms]
Tiempo TOTAL:  19287.799 [ms]


---
# 4 Tabla de pasos


 Procesador | Funciòn | Detalle
------------|---------|----------
CPU      | pip install pycuda    | Instala en el cuaderno los driver de CUDA para Python.
CPU      |  import                | Importa los módulos para funcionar.
CPU      |  datetime.now()        | Toma el tiempo actual.
CPU      |  numpy.random(n)       | Crear una vector con numeros ramdoms de n elementos.
CPU      |  ar.astype(np.int32()) | Convierte los elemementos del array ar en elementos de tipo int 32 bit.
**GPU**  |  cuda.mem_alloc()      | Reserva la memoria para el vector en la GPU.
**GPU**  |  cuda.memcpy_htod()    | Copia los valores en crudo del vector al GPU.
CPU      |  SourceModule()        | Posee el còdigo del kernel.
CPU      |  module.get_function() | convierte el texto del kernel en funcion de Python.
CPU      |  dim_hilo_x            | Calcula la dimension para la ejecuciòn de 1D.
**GPU**  |  kernel()              | Ejecuta el kernel en GPU, enviando los parametros.
CPU      |  print()               | Informa medieante un mensaje.
CPU      | cuda.memcpy_dtoh()     | Copia desde la memoria GPU al CPU.



---
# 5 Conclusiones
Se hicieron varias pruebas con vectores de un tamaño pequeño (50 elementos), y con un tamaño grande (100000000 elementos). Se observó que para vectores pequeños el tiempo de ejecución de la forma secuencial utilizando la CPU y el tiempo de ejecución de la forma paralela utilizando la GPU, son aproximadamente iguales y en ocasiones el tiempo de ejecución total de la GPU es mayor que el tiempo de ejecución total de la CPU, parecería que no estaría mejorando el tiempo de ejecución utilizando la GPU. Sin embargo al momento de tener vectores con muchos elementos se observó que el tiempo de ejecución de la búsqueda de un elemento utilizando la GPU se mantiene constante, aproximadamente, mientras que el tiempo de ejecución de la búsqueda de un elemento utilizando la CPU aumenta significativamente cada vez que aumenta la cantidad de elementos del vector. Se concluye que la utilización de la GPU baja el tiempo de ejecución de la búsqueda de un elemento en comparación con la CPU siempre que se tiene un vector de tamaño considerable por ejemplo 1000.

---
# 6 Bibliografía

[1] Desarrollo Teorico algoritmo de busqueda: [Pagina Wiki](https://es.wikipedia.org/wiki/Algoritmo_de_b%C3%BAsqueda#:~:text=Un%20algoritmo%20de%20b%C3%BAsqueda%20es,en%20una%20partida%20de%20ajedrez.)

[2] Introducción a Python: [Página Colab](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb) 

[3] Algoritmo de Busqueda y Ordenamiento: [PDF](https://www.inf.utfsm.cl/~noell/IWI-131-p1/Tema8b.pdf)
